<a href="https://colab.research.google.com/github/Miragekaefer/NewGameIdea/blob/main/Capstone_Project_TOD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Project Geospace Maschine Learnig Model**

**Content:
Building a geospace Model to recognize different Usages of Land**

(description of our project / Reasons for choosing this project topic (City Planning through land usage recocnition)

The initial phase in constructing a spatial-based model involves the collection of essential image data. To achieve this, we opted to use OpenStreetMaps, where images are pre-defined based on building types, road types, and more. This eliminates the need for manual scripting of every entity within each image, allowing us to concentrate on tasking of masking images for the learning process.

For obtaining these images locally, we utilized a Google download function, necessitating the use of the Google Cloud service and a dedicated API key.

In [ ]:
# function to donwload images from the googlestaticmap api
def download_images(coordinates, zoom=16):
    # define the request parameters
    url = "https://maps.googleapis.com/maps/api/staticmap?"
    api_key = ""
    size = "640x640"
    scale = "1"
    maptype = "satellite"

    # pose get request
    response = requests.get(url + "center=" + str(coordinates[0]) + "," + str(coordinates[1]) + "&zoom=" + str(zoom) + "&size=" + size + "&maptype=" + maptype + "&scale=" + scale + "&sensor=false" + "&key=" + api_key, stream=True,)
    #stream repsonse into file and save it
    with open(os.getcwd() + os.sep + "Data" + os.sep + "Images" + os.sep + str(coordinates[0]) + "_" + str(coordinates[1]) + ".png", "wb",) as out_file:
        shutil.copyfileobj(response.raw, out_file)
    #delete the repsonse arifact
    del response

Prior to utilizing the image downloader, we decided to predefine specific areas and coordinates. This precautionary step ensures that we exclude images from regions that are of no importance for the training model, such as images from oceans or seas.

Our approach involved taking a package that contains a list of all cities available in OpenStreetMaps, each with a population of 15,000 or more. This strategy was implemented to avoid irrelevant images and place a greater emphasis on the initial concept of city analysis.

In [ ]:
# load dataset of all cities in the world with >15000 inhabitants
package = Package("https://datahub.io/core/world-cities/datapackage.json")

cities_list = package.get_resource("world-cities_csv").read()

In [ ]:
# fetch random city contained in the dataset
def get_random_city():

    city = random.choice(cities_list)

    return city

The coordinates are required in establishing the length and width of each image, ensuring equality in size across all images. The following function is designed to define these coordinates based on the nodes of the OpenStreetMap image.

In [ ]:
def get_coordinates_from_id(osmid):

    # connect to overpass api endpoind
    api = overpy.Overpass()

    # query the api and save the response
    result = api.query(f"node({osmid});out;")

    for node in result.nodes:
        # extract coordinates from response
        coordinates = (float(node.lat), float(node.lon))

    # if coordinates are available return them
    try:
        return coordinates

    #else print error
    except:
        print(f"No coordinates for {osmid}")


To further suit our images for the training process, we check their compatibility by scaling them to a standardized size commonly used for map image data.

In [ ]:
def latLngToPoint(mapWidth, mapHeight, lat, lng):
    x = (lng + 180) * (mapWidth / 360)

    y = ((1 - math.log(math.tan(lat * math.pi / 180) + 1 / math.cos(lat * math.pi / 180)) / math.pi) / 2) * mapHeight

    return (x, y)



def pointToLatLng(mapWidth, mapHeight, x, y):
    lng = x / mapWidth * 360 - 180
    n = math.pi - 2 * math.pi * y / mapHeight
    lat = 180 / math.pi * math.atan(0.5 * (math.exp(n) - math.exp(-n)))


    return (lat, lng)



def getImageBounds(lat, lng, zoom):
    picHeight = 640
    picWidth = 640


    mapHeight = 256
    mapWidth = 256


    xScale = math.pow(2, zoom) / (picWidth / mapWidth)
    yScale = math.pow(2, zoom) / (picHeight / mapWidth)


    centreX, centreY = latLngToPoint(mapWidth, mapHeight, lat, lng)


    southWestX = centreX - (mapWidth / 2) / xScale
    southWestY = centreY + (mapHeight / 2) / yScale
    SWlat, SWlng = pointToLatLng(mapWidth, mapHeight, southWestX, southWestY)


    northEastX = centreX + (mapWidth / 2) / xScale
    northEastY = centreY - (mapHeight / 2) / yScale
    NElat, NElng = pointToLatLng(mapWidth, mapHeight, northEastX, northEastY)


    return [SWlat, SWlng, NElat, NElng]

With all nessessary functions established, the next step involves defining a list that includes all relevant land-use types. Since each node of the image data has an assigned type, we can employ a pre-defined set for land usages to ease this process.

In [ ]:
#list with all landuses
dictionary_landuse = [
    "commercial",
    "construction",
    "education",
    "fairground",
    "industrial",
    "residential",
    "retail",
    "institutional",
    "aquaculture",
    "allotments",
    "farmland",
    "farmyard",
    "paddy",
    "animal_keeping",
    "flowerbed",
    "forest",
    "greenhouse_horticulture",
    "meadow",
    "orchard",
    "plant_nursery",
    "vineyard",
    "basin",
    "salt_pond",
    "brownfield",
    "cemetery",
    "depot",
    "garages",
    "grass",
    "greenfield",
    "landfill",
    "military",
    "port",
    "quarry",
    "railway",
    "recreation_ground",
    "religious",
    "village_green",
    "winter_sports",
]

Our decision was to train the model on 5000 images.

In [ ]:
# function to generate a set of images for n cities
def get_random_samples(n=1):
    # loop to do multiple cities

    for x in range(n):

        # get a random city
        city = get_random_city()

        # get the coordinates for one of all the different landuses
        coordinates = get_coordinates_for_each_landuse(city)

        try:

            # loop over each coordinate
            for coordinate in coordinates:

                try:
                    # check if data is actually present in the given area
                    south, east, north, west = getImageBounds(
                        float(coordinate[0]), float(coordinate[1]), 16
                    )
                    landuse = ox.features_from_bbox(
                        north, south, east, west, tags={"landuse": True}
                    )
                    # download the image of the given coordinate
                    download_images(coordinate)

                except:
                    print("No landuse data was found")

        except:
            print("Coordinates were returned empty")

In [ ]:
# initial count of how much data is already in the folder
datanames = os.listdir(os.getcwd() + os.sep + "Data" + os.sep + "Images")

# loop as long as less then 5000 examples
while len(datanames) < 5000:

    get_random_samples()

    datanames = os.listdir(os.getcwd() + os.sep + "Data" + os.sep + "Images")